In [26]:
import tensorflow as tf
# import tensorflow_hub as hub
import os
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
#from keras import optimizers
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
# import matplotlib.pyplot as plt
from glob import glob

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [27]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


In [28]:
#Set the image size as per the VGG16 archetecture
IMAGE_SIZE = [224, 224]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls '/content/drive'

MyDrive


In [29]:
#Give dataset path
train_path = 'X:\\Projects\\tomato\\train'
validation_path = 'X:\\Projects\\tomato\\val'
test_path = 'X:\\Projects\\tomato\\test'

In [30]:
import time
import os
from os.path import exists

def count(dir, counter=0):
    "returns number of files in dir and subdirs"
    for pack in os.walk(dir):
        for f in pack[2]:
            counter += 1
    return dir + " : " + str(counter) + "files"

In [31]:
print('total images for training :', count(train_path))
print('total images for validation :', count(validation_path))
print('total images for validation :', count(test_path))

total images for training : X:\Projects\tomato\train : 6097files
total images for validation : X:\Projects\tomato\val : 871files
total images for validation : X:\Projects\tomato\test : 1744files


In [32]:
folders = glob('X:\\Projects\\tomato\\train\\*')
print(len(folders))


10


In [33]:
BATCH_SIZE = 64

train_generator = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.1) # VGG16 preprocessing

validation_generator = ImageDataGenerator(preprocessing_function=preprocess_input) 

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessin

In [34]:
class_subset = sorted(os.listdir('X:\\Projects\\tomato\\train'))
print(class_subset)
traingen = train_generator.flow_from_directory(train_path,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=5)

validgen = train_generator.flow_from_directory(train_path,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=4)

testgen = test_generator.flow_from_directory(test_path,
                                             target_size=(224, 224),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=4)

['bacterialspot', 'earlyblight', 'healthy', 'lateblight', 'leafmold', 'mosaicvirus', 'septorialeafspot', 'spidermites', 'targetspot', 'yellowleafcurlvirus']
Found 5488 images belonging to 10 classes.
Found 609 images belonging to 10 classes.
Found 1744 images belonging to 10 classes.


# **VGG16**

In [35]:
from keras.applications.vgg16 import VGG16
model1 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [36]:
for layer in model1.layers:
  layer.trainable = False

In [37]:
model1.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [38]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
batnorm = layers.BatchNormalization()
dropout = layers.Dropout(0.20)
prediction_layer = layers.Dense(10, activation='sigmoid')


vgg16 = models.Sequential([
    model1,
    flatten_layer,
    batnorm,
    dropout,
    prediction_layer
])

In [39]:
vgg16.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25088)             100352    
_________________________________________________________________
dropout_2 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                250890    
Total params: 15,065,930
Trainable params: 301,066
Non-trainable params: 14,764,864
_________________________________________________________________


In [40]:
optimizer=Adam(learning_rate=0.001)
vgg16.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='X:\\Projects\\tomato\\vgg16.h5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


vgg_history = vgg16.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1], 
                            verbose=1)

Epoch 1/50
85/85 [==============================] - 40s 456ms/step - loss: 0.6806 - accuracy: 0.7972 - val_loss: 0.4839 - val_accuracy: 0.8785

Epoch 00001: val_accuracy improved from -inf to 0.87847, saving model to X:\Projects\tomato\vgg16.h5
Epoch 2/50
85/85 [==============================] - 39s 462ms/step - loss: 0.0520 - accuracy: 0.9836 - val_loss: 0.3953 - val_accuracy: 0.8924

Epoch 00002: val_accuracy improved from 0.87847 to 0.89236, saving model to X:\Projects\tomato\vgg16.h5
Epoch 3/50
85/85 [==============================] - 41s 486ms/step - loss: 0.0162 - accuracy: 0.9972 - val_loss: 0.3782 - val_accuracy: 0.9097

Epoch 00003: val_accuracy improved from 0.89236 to 0.90972, saving model to X:\Projects\tomato\vgg16.h5
Epoch 4/50
85/85 [==============================] - 42s 496ms/step - loss: 0.0077 - accuracy: 0.9993 - val_loss: 0.3487 - val_accuracy: 0.9167

Epoch 00004: val_accuracy improved from 0.90972 to 0.91667, saving model to X:\Projects\tomato\vgg16.h5
Epoch 5/50



Epoch 00040: val_accuracy did not improve from 0.92535
Epoch 41/50
85/85 [==============================] - 45s 524ms/step - loss: 0.0131 - accuracy: 0.9967 - val_loss: 1.0836 - val_accuracy: 0.9062

Epoch 00041: val_accuracy did not improve from 0.92535
Epoch 42/50
85/85 [==============================] - 44s 521ms/step - loss: 0.0042 - accuracy: 0.9982 - val_loss: 1.1199 - val_accuracy: 0.9080

Epoch 00042: val_accuracy did not improve from 0.92535
Epoch 43/50
85/85 [==============================] - 44s 517ms/step - loss: 0.0046 - accuracy: 0.9985 - val_loss: 1.0794 - val_accuracy: 0.9149

Epoch 00043: val_accuracy did not improve from 0.92535
Epoch 44/50
85/85 [==============================] - 45s 527ms/step - loss: 0.0070 - accuracy: 0.9974 - val_loss: 0.9475 - val_accuracy: 0.9062

Epoch 00044: val_accuracy did not improve from 0.92535
Epoch 45/50
85/85 [==============================] - 45s 527ms/step - loss: 0.0054 - accuracy: 0.9982 - val_loss: 0.9556 - val_accuracy: 0.9219


In [41]:
import numpy as np
# Generate predictions
vgg16.load_weights('X:\\Projects\\tomato\\vgg16.h5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg16.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [42]:
from sklearn.metrics import accuracy_score

vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

VGG16 Model Accuracy without Fine-Tuning: 90.71%


In [43]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(true_classes,vgg_pred_classes))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83        78
           1       0.86      0.82      0.84       200
           2       0.98      0.96      0.97       200
           3       0.89      0.87      0.88       200
           4       0.95      0.93      0.94       191
           5       0.91      0.96      0.94        75
           6       0.87      0.84      0.86       200
           7       0.92      0.94      0.93       200
           8       0.88      0.91      0.89       200
           9       0.95      0.99      0.97       200

    accuracy                           0.91      1744
   macro avg       0.90      0.91      0.90      1744
weighted avg       0.91      0.91      0.91      1744



In [44]:
print(confusion_matrix(true_classes,vgg_pred_classes))

[[ 65   5   0   1   0   0   2   1   0   4]
 [  5 165   1  13   3   0   6   1   3   3]
 [  0   1 193   0   0   0   2   0   4   0]
 [  0  13   1 174   4   0   4   1   2   1]
 [  0   1   0   4 177   2   5   2   0   0]
 [  0   0   0   0   0  72   3   0   0   0]
 [  6   4   1   4   3   2 169   2   8   1]
 [  0   0   0   0   0   3   0 187   9   1]
 [  2   2   1   0   0   0   4   8 182   1]
 [  0   1   0   0   0   0   0   1   0 198]]


# **ResNet152**

In [45]:
from keras.models import Model
import keras
model6 = tf.keras.applications.ResNet152(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in model6.layers:
    layer.trainable = False

In [46]:
model6.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [47]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
batnorm = layers.BatchNormalization()
dropout = layers.Dropout(0.20)
prediction_layer = layers.Dense(10, activation='sigmoid')

resnet152 = models.Sequential([
    model6,
    flatten_layer,
    batnorm,
    dropout,
    prediction_layer
])

In [48]:
resnet152.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 7, 7, 2048)        58370944  
_________________________________________________________________
flatten_3 (Flatten)          (None, 100352)            0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 100352)            401408    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100352)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1003530   
Total params: 59,775,882
Trainable params: 1,204,234
Non-trainable params: 58,571,648
_________________________________________________________________


In [49]:
optimizer=Adam(learning_rate=0.001)
resnet152.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint, EarlyStopping
optim_1 = Adam(learning_rate=0.001)
#n_classes=3

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='X:\\Projects\\tomato\\resnet152.h5',
                                  save_best_only=True,
                                  monitor='val_accuracy',
                                  mode='max',
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')


vgg_history = resnet152.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1], 
                            verbose=1)

Epoch 1/50
85/85 [==============================] - 86s 884ms/step - loss: 1.1329 - accuracy: 0.8282 - val_loss: 0.6960 - val_accuracy: 0.9028

Epoch 00001: val_accuracy improved from -inf to 0.90278, saving model to X:\Projects\tomato\resnet152.h5


C:\Users\bhala\anaconda3\envs\NNDL\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/50
85/85 [==============================] - 73s 859ms/step - loss: 0.2257 - accuracy: 0.9622 - val_loss: 0.6791 - val_accuracy: 0.8976

Epoch 00002: val_accuracy did not improve from 0.90278
Epoch 3/50
85/85 [==============================] - 75s 887ms/step - loss: 0.0977 - accuracy: 0.9836 - val_loss: 0.7878 - val_accuracy: 0.9236

Epoch 00003: val_accuracy improved from 0.90278 to 0.92361, saving model to X:\Projects\tomato\resnet152.h5
Epoch 4/50
85/85 [==============================] - 74s 870ms/step - loss: 0.0681 - accuracy: 0.9880 - val_loss: 0.9097 - val_accuracy: 0.9288

Epoch 00004: val_accuracy improved from 0.92361 to 0.92882, saving model to X:\Projects\tomato\resnet152.h5
Epoch 5/50
85/85 [==============================] - 74s 874ms/step - loss: 0.0388 - accuracy: 0.9924 - val_loss: 0.8731 - val_accuracy: 0.9219

Epoch 00005: val_accuracy did not improve from 0.92882
Epoch 6/50
85/85 [==============================] - 76s 890ms/step - loss: 0.0449 - accuracy: 0.99


Epoch 00040: val_accuracy did not improve from 0.94965
Epoch 41/50
85/85 [==============================] - 78s 912ms/step - loss: 0.0350 - accuracy: 0.9971 - val_loss: 1.9634 - val_accuracy: 0.9410

Epoch 00041: val_accuracy did not improve from 0.94965
Epoch 42/50
85/85 [==============================] - 77s 910ms/step - loss: 0.0241 - accuracy: 0.9967 - val_loss: 1.9349 - val_accuracy: 0.9444

Epoch 00042: val_accuracy did not improve from 0.94965
Epoch 43/50
85/85 [==============================] - 77s 906ms/step - loss: 0.0673 - accuracy: 0.9958 - val_loss: 2.3579 - val_accuracy: 0.9253

Epoch 00043: val_accuracy did not improve from 0.94965
Epoch 44/50
85/85 [==============================] - 77s 905ms/step - loss: 0.0338 - accuracy: 0.9969 - val_loss: 2.1273 - val_accuracy: 0.9271

Epoch 00044: val_accuracy did not improve from 0.94965
Epoch 45/50
85/85 [==============================] - 76s 899ms/step - loss: 0.0533 - accuracy: 0.9954 - val_loss: 2.0395 - val_accuracy: 0.9358


In [50]:
import numpy as np
# Generate predictions
resnet152= tf.keras.models.load_model('X:\\Projects\\tomato\\resnet152.h5') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

resnet152_preds = resnet152.predict(testgen)
resnet152_pred_classes = np.argmax(resnet152_preds, axis=1)

In [51]:
from sklearn.metrics import accuracy_score

resnet152_acc = accuracy_score(true_classes, resnet152_pred_classes)
print("ResNet152 Model Accuracy without Fine-Tuning: {:.2f}%".format(resnet152_acc * 100))

ResNet152 Model Accuracy without Fine-Tuning: 39.39%


In [52]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(true_classes,resnet152_pred_classes))

              precision    recall  f1-score   support

           0       0.24      1.00      0.38        78
           1       0.30      0.67      0.41       200
           2       0.54      0.84      0.66       200
           3       0.37      0.36      0.37       200
           4       0.40      0.46      0.43       191
           5       0.41      0.33      0.37        75
           6       0.42      0.14      0.20       200
           7       0.75      0.28      0.40       200
           8       1.00      0.13      0.23       200
           9       1.00      0.07      0.12       200

    accuracy                           0.39      1744
   macro avg       0.54      0.43      0.36      1744
weighted avg       0.57      0.39      0.35      1744



In [53]:
confusion_matrix(true_classes,resnet152_pred_classes)

array([[ 78,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 66, 133,   0,   1,   0,   0,   0,   0,   0,   0],
       [ 16,  15, 169,   0,   0,   0,   0,   0,   0,   0],
       [ 44,  66,  17,  73,   0,   0,   0,   0,   0,   0],
       [  6,  37,  13,  47,  88,   0,   0,   0,   0,   0],
       [  0,  10,   7,   2,  31,  25,   0,   0,   0,   0],
       [ 68,  44,  11,  22,  22,   6,  27,   0,   0,   0],
       [  5,  26,  31,  33,  28,  13,   9,  55,   0,   0],
       [ 28,  40,  48,   7,  11,   8,  18,  14,  26,   0],
       [ 20,  75,  18,  10,  41,   9,  10,   4,   0,  13]], dtype=int64)